In [5]:
import ast
import inspect
from lib.logging import logger

In [35]:
import time
from graph import Graph, GraphNode

In [27]:
class Node:
    def __init__(self, settable=True):
        self._settable = settable
        self._cachedValue = None

    def __call__(self, func):
        
        def wrapper(*args, **kwargs):
            if not self._cachedValue:
                self._cachedValue = func(*args, **kwargs)
            return self._cachedValue
        
        if self._settable:
            def setter(value):
                logger.info(f'Setting value of Node: {func.__name__} = {value}')
                self._cachedValue = value
            wrapper.set = setter

        def delete():
            logger.info(f'Clearing value of Node: {func.__name__}')
            self._cachedValue = None
        wrapper.delete = delete
        
        return wrapper


@Node(settable=True)
def A():
    logger.info('expensive fn called')
    time.sleep(2)
    return 1

In [40]:
class Test:
    def __init__(self):
        self.a = 'a'

    def A(self):
        return 1

node = GraphNode(Test.A, settable=True)

In [ ]:
Test.A.__get__()

<method-wrapper '__get__' of function object at 0x000001941261ACA0>

TypeError: hash() takes exactly one argument (2 given)

In [6]:
from graph import MyGraph, graph_node
g = MyGraph()

2025-08-11 22:45:34.650India Standard Time 20828 DEBUG [MyGraph] __init__ start
2025-08-11 22:45:34.653India Standard Time 20828 DEBUG [Graph] __init__ start
2025-08-11 22:45:34.654India Standard Time 20828 DEBUG [Graph] __init__ end
2025-08-11 22:45:34.655India Standard Time 20828 DEBUG [MyGraph] __init__ end


In [9]:
g.A.set

<function graph.GraphNode.__get__.<locals>.setter(value)>

In [3]:
dg = g.getDependencyGraph()
dg

{'A': set(),
 'B': {'A'},
 'C': {'A', 'B'},
 'D': {'A', 'B'},
 'E': {'A', 'B', 'D'},
 'F': {'A', 'B', 'D', 'E'}}

In [4]:
g.getReverseDependencyGraph()

{'A': {'B', 'C', 'D', 'E', 'F'},
 'B': {'C', 'D', 'E', 'F'},
 'D': {'E', 'F'},
 'E': {'F'}}

In [128]:
src = inspect.getsource(g.__class__)
print(src)

class MyGraph(Graph):
    def __init__(self):
        logger.debug('[MyGraph] __init__ start')
        super().__init__()
        logger.debug('[MyGraph] __init__ end')

    @graph_node(settable=True)
    def A(self):
        logger.debug('[MyGraph] graph_node A called')
        return 10

    @graph_node()
    def B(self):
        logger.debug('[MyGraph] graph_node B called')
        return self.A() ** 2

    @graph_node()
    def C(self):
        logger.debug('[MyGraph] graph_node C called')
        return self.B() + self.A()
    
    @graph_node()
    def D(self, value):
        logger.debug('[MyGraph] graph_node D called')
        return value*self.B()
    
    @graph_node()
    def E(self):
        logger.debug('[MyGraph] graph_node E called')
        return self.D( self.A() ) * 2
    
    @graph_node()
    def F(self):
        logger.debug('[MyGraph] graph_node F called')
        if self.A() > 5:
            return self.E()*1.5
        else:
            return self.D(5)



In [129]:
tree = ast.parse(src)

In [32]:
nodes = []
for node in ast.walk(tree):
    if isinstance(node, ast.FunctionDef):
        nodes.append(node)

In [51]:
b = nodes[2]

In [99]:
for decor in b.decorator_list:
    if decor.func.id == 'graph_node':
        print(True)

True


In [137]:
from collections import deque

def expand_dependencies_optimized(graph):
    graph = {k: set(v) for k, v in graph.items()}        
    nodesToExpand = deque(graph.keys())

    while nodesToExpand:
        node = nodesToExpand.popleft()
        current_deps = graph[node].copy()

        for dep in current_deps:
            new_items = graph[dep].difference(current_deps)
            if not new_items:
                continue
            graph[node].update(new_items)
            nodesToExpand.append(node)
    
    return graph

In [138]:
def getDependencyGraphFromTree(tree):
    dependency_graph = {}
    for node in ast.walk(tree):
        if not any(decor.func.id == graph_node.__name__ for decor in getattr(node, 'decorator_list', [])):
            continue
        dependency_graph[node.name] = set()
        for inner_node in node.body:
            for _n in ast.walk(inner_node):
                if isinstance(_n, ast.Attribute) and _n.value.id=='self':
                    dependency_graph[node.name].add(_n.attr)

    return dependency_graph

def getDependencyGraphFromClass(cls):
    src = inspect.getsource(cls)
    tree = ast.parse(src)
    return getDependencyGraphFromTree(tree)

In [139]:
dep = getDependencyGraphFromClass(g.__class__)
dep

{'A': set(),
 'B': {'A'},
 'C': {'A', 'B'},
 'D': {'B'},
 'E': {'A', 'D'},
 'F': {'A', 'D', 'E'}}

In [140]:
expand_dependencies_optimized(dep)

{'A': set(),
 'B': {'A'},
 'C': {'A', 'B'},
 'D': {'A', 'B'},
 'E': {'A', 'B', 'D'},
 'F': {'A', 'B', 'D', 'E'}}

In [113]:
for node in ast.walk(tree):
    if not any(decor.func.id == 'graph_node' for decor in getattr(node, 'decorator_list', [])):
        continue
    print(node.name, '--------------')
    for inner_node in node.body:
        print('\t', inner_node.__class__.__name__)
        for _n in ast.walk(inner_node):
            if isinstance(_n, ast.Attribute):
                print('\t\t', _n.value.id, _n.attr)


A --------------
	 Expr
		 logger debug
	 Return
B --------------
	 Expr
		 logger debug
	 Return
		 self A
C --------------
	 Expr
		 logger debug
	 Return
		 self B
		 self A
D --------------
	 Expr
		 logger debug
	 Return
		 self B
E --------------
	 Expr
		 logger debug
	 Return
		 self D
		 self A
F --------------
	 Expr
		 logger debug
	 If
		 self A
		 self D
		 self E


In [108]:
inner_node.orelse

In [55]:
b_body = b.body[0]
b_return = b.body[1]

In [104]:
b_return.__class__.__name__

'Return'

In [ ]:
b_body.value.func.value.

In [65]:
for n in ast.walk(b.body[1]):
    if isinstance(n, ast.Attribute):
        print(n.attr)

A


In [77]:
baa = b.args.args[0]

In [96]:
b_return.value.left.func.value.id

'self'

In [29]:
# Pretty-print AST
astpretty.pprint(tree)

Module(
    body=[
        ClassDef(
            lineno=1,
            col_offset=0,
            end_lineno=38,
            end_col_offset=21,
            name='MyGraph',
            bases=[Name(lineno=1, col_offset=14, end_lineno=1, end_col_offset=19, id='Graph', ctx=Load())],
            keywords=[],
            body=[
                FunctionDef(
                    lineno=2,
                    col_offset=4,
                    end_lineno=5,
                    end_col_offset=46,
                    name='__init__',
                    args=arguments(
                        posonlyargs=[],
                        args=[arg(lineno=2, col_offset=17, end_lineno=2, end_col_offset=21, arg='self', annotation=None, type_comment=None)],
                        vararg=None,
                        kwonlyargs=[],
                        kw_defaults=[],
                        kwarg=None,
                        defaults=[],
                    ),
                    body=[
                 

In [25]:
import astpretty

In [19]:
from graphviz import Digraph
import pprint

In [97]:
def build_graph(node, graph=None, parent=None):
    if graph is None:
        graph = Digraph()
    node_id = str(id(node))
    
    label_pre = ''
    if isinstance(node, ast.FunctionDef):
        label_pre = f'def {node.name}()'
    if isinstance(node, ast.Attribute):
        if hasattr(node.value, 'id'):
            label_pre = f'{node.value.id}.{node.attr}'
        else:
            label_pre = {node.attr}
    label = f'{label_pre}_{type(node).__name__}' if label_pre else type(node).__name__
    
    graph.node(node_id, label)
    if parent:
        graph.edge(parent, node_id)

    for child in ast.iter_child_nodes(node):
        build_graph(child, graph, node_id)
    return graph

# source_code = "x = a + b"
# tree = ast.parse(source_code)

dot = build_graph(tree)
dot.render("ast_tree", format="png", cleanup=True)  # creates ast_tree.pn

2025-08-11 00:30:30.952India Standard Time 31684 DEBUG write lines to 'ast_tree'
2025-08-11 00:30:30.957India Standard Time 31684 DEBUG run [WindowsPath('dot'), '-Kdot', '-Tpng', '-O', 'ast_tree']
2025-08-11 00:30:31.730India Standard Time 31684 DEBUG delete 'ast_tree'


'ast_tree.png'

In [67]:
type(b).__name__

'FunctionDef'